### Import packages

In [3]:
import torch, torchvision
import torchvision.transforms as transforms
import cv2
import os
from os import path
import json

### Extract frames from videos and place them inside video name folder

In [5]:
# def extract_frames(file_path, target_dir):
#     if not os.path.exists(target_dir):
#         os.makedirs(target_dir)

#     vidcap = cv2.VideoCapture(file_path)
#     success, image = vidcap.read()
#     count = 0
 
#     while success:
#       frame_path = os.path.join(target_dir, f'{count}.jpg')
#       cv2.imwrite(frame_path, image)     # save frame as JPEG file
#       count += 1
#       success, image = vidcap.read()
#       success, image = vidcap.read()
#       success, image = vidcap.read()


# root = 'D:\Documentos\INESC\hmdb51_org'
# frame_root = 'D:\Documentos\INESC\hmdb51_frames'
# labels_path = 'D:\Documentos\INESC\labels.csv'
# class_name_to_label_path = 'D:\Documentos\INESC\class_name_to_label.json'

# # read files
# files = []

# for class_name in os.listdir(root):
#     for video_name in os.listdir(os.path.join(root, class_name)):
#         files.append([os.path.join(class_name, video_name), class_name])

# # normalize labels
# class_name_to_label = {}
# current_label = -1

# for vid in files:
#     label = class_name_to_label.get(vid[1], -1)

#     if label == -1:
#         current_label += 1
#         class_name_to_label[vid[1]] = current_label
#         label = current_label

#     vid[1] = label


# # save file paths
# if not os.path.exists(os.path.split(labels_path)[0]):
#     os.makedirs(os.path.split(labels_path)[0])

# f = open(labels_path, 'w')

# f.write('path,label\n')

# for vid in files:
#     f.write(f'{vid[0]},{vid[1]}\n')

# f.close()

# # save label normalization
# if not os.path.exists(os.path.split(class_name_to_label_path)[0]):
#     os.makedirs(os.path.split(class_name_to_label_path)[0])

# with open(class_name_to_label_path, 'w') as json_file:
#     json.dump(class_name_to_label, json_file, indent=4)

# # extract frames
# for i, vid in enumerate(files):
#     file_path = os.path.join(root, vid[0])
#     target_dir = os.path.join(frame_root, vid[0])

#     extract_frames(file_path, target_dir)

#     print(f'{i+1}/{len(files)}')


1/5689
2/5689
3/5689
4/5689
5/5689
6/5689
7/5689
8/5689
9/5689
10/5689
11/5689
12/5689
13/5689
14/5689
15/5689
16/5689
17/5689
18/5689
19/5689
20/5689
21/5689
22/5689
23/5689
24/5689
25/5689
26/5689
27/5689
28/5689
29/5689
30/5689
31/5689
32/5689
33/5689
34/5689
35/5689
36/5689
37/5689
38/5689
39/5689
40/5689
41/5689
42/5689
43/5689
44/5689
45/5689
46/5689
47/5689
48/5689
49/5689
50/5689
51/5689
52/5689
53/5689
54/5689
55/5689
56/5689
57/5689
58/5689
59/5689
60/5689
61/5689
62/5689
63/5689
64/5689
65/5689
66/5689
67/5689
68/5689
69/5689
70/5689
71/5689
72/5689
73/5689
74/5689
75/5689
76/5689
77/5689
78/5689
79/5689
80/5689
81/5689
82/5689
83/5689
84/5689
85/5689
86/5689
87/5689
88/5689
89/5689
90/5689
91/5689
92/5689
93/5689
94/5689
95/5689
96/5689
97/5689
98/5689
99/5689
100/5689
101/5689
102/5689
103/5689
104/5689
105/5689
106/5689
107/5689
108/5689
109/5689
110/5689
111/5689
112/5689
113/5689
114/5689
115/5689
116/5689
117/5689
118/5689
119/5689
120/5689
121/5689
122/5689
123/5689
1

### Split labels csv file into train and validate - 80/20

In [44]:
import pandas as pd

def split(label):
    path = 'D:/Documentos/INESC'
    train = label.sample(frac=0.8, random_state=201)
    val = label.drop(train.index)
    train.to_csv(path + '/train.csv', mode='a', header=False)
    val.to_csv(path + '/val.csv', mode='a', header=False)
    

labels_path = 'D:\Documentos\INESC\labels.csv'
labels_data = pd.read_csv(labels_path)
last_label = labels_data.tail(1)['label']
last_label = last_label.iloc[0]
max_labels = labels_data.nunique()
max_labels= max_labels['label']
index = 0

for i in range(max_labels):
    label = labels_data[labels_data['label'] == index]
    split(label)
    index +=1 


### Load the images to a model